# Chapter 13

Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [2]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [3]:
import thinkstats2
import thinkplot
import numpy as np
import pandas as pd

In [4]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist

In [5]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

import nsfg

preg = nsfg.ReadFemPreg()
complete = preg.query("outcome in [1, 3, 4]").prglngth
cdf = thinkstats2.Cdf(complete, label="cdf")

In [6]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/survival.py")

import survival


def MakeSurvivalFromCdf(cdf, label=""):
    """Makes a survival function based on a CDF.

    cdf: Cdf

    returns: SurvivalFunction
    """
    ts = cdf.xs
    ss = 1 - cdf.ps
    return survival.SurvivalFunction(ts, ss, label)

sf = MakeSurvivalFromCdf(cdf, label="survival")
hf = sf.MakeHazardFunction(label="hazard")



Downloaded survival.py


/Users/madisonchristiansen/survival.py:71: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=self.ts)


In [10]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz"
)

resp6 = nsfg.ReadFemResp()

In [8]:
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/1995FemRespData.dat.gz"
)
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2006_2010_FemRespSetup.dct"
)
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2006_2010_FemResp.dat.gz"
)

resp5 = survival.ReadFemResp1995()
resp6 = survival.ReadFemResp2002()
resp7 = survival.ReadFemResp2010()

resps = [resp5, resp6, resp7]

Downloaded 1995FemRespData.dat.gz
Downloaded 2006_2010_FemRespSetup.dct
Downloaded 2006_2010_FemResp.dat.gz


## Exercises

**Exercise:**    In NSFG Cycles 6 and 7, the variable `cmdivorcx` contains the date of divorce for the respondent’s first marriage, if applicable, encoded in century-months.

Compute the duration of marriages that have ended in divorce, and the duration, so far, of marriages that are ongoing. Estimate the hazard and survival curve for the duration of marriage.

Use resampling to take into account sampling weights, and plot data from several resamples to visualize sampling error.

Consider dividing the respondents into groups by decade of birth, and possibly by age at first marriage.

In [12]:
from __future__ import print_function

import pandas
import numpy as np

import thinkplot
import thinkstats2
import survival

In [33]:
def clean(resp): 
    resp.cmdivorcx.replace(np.nan, inplace=True)

    resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)
    resp['duration'] = (resp.cmdivorcx - resp.cmmarrhx) 
    resp['durationsofar'] = (resp.cmintvw - resp.cmmarrhx) 

    month0 = pandas.to_datetime('1899-12-15')
    dates = [month0 + pandas.DateOffset(months=cm) 
             for cm in resp.cmbirth]
    resp['decade'] = (pandas.DatetimeIndex(dates).year - 1900) 

In [34]:
def ResampDivorceCurve(resps):
    for _ in range(41):
        samples = [thinkstats2.ResampleRowsWeighted(resp) 
                   for resp in resps]
        sample = pandas.concat(samples, ignore_index=True)

In [35]:
def ResampDivorceCurvebyDecade(resps):
    for i in range(41):
        samples = [thinkstats2.ResampleRowsWeighted(resp) 
                   for resp in resps]
        sample = pandas.concat(samples, ignore_index=True)
        groups = sample.groupby('decade')

        EstSurvivalbyDecade(groups, alpha=0.1)

In [36]:
def EstSurvivalbyDecade(groups, **options):
    thinkplot.PrePlot(len(groups))
    for name, group in groups:
        print(name, len(group))
        _, sf = EstSurvival(group)

In [37]:
def EstSurvival(resp):
    complete = resp[resp.notdivorced == 0].duration.dropna()
    ongoing = resp[resp.notdivorced == 1].durationsofar.dropna()
    hf = survival.EstimateHazardFunction(complete, ongoing)
    sf = hf.MakeSurvival()

    return hf, sf

In [38]:
resp6 = survival.ReadFemResp2002()
clean(resp6)
married6 = resp6[resp6.evrmarry==1]

resp7 = survival.ReadFemResp2010()
clean(resp7)
married7 = resp7[resp7.evrmarry==1]

ResampDivorceCurvebyDecade([married6, married7])

57 69
58 258
59 210
60 265
61 271
62 367
63 377
64 473
65 470
66 475
67 543
68 527
69 530
70 475
71 476
72 400
73 382
74 363
75 351
76 332
77 283
78 323
79 311
80 254
81 257
82 167
83 122
84 106
85 62
86 64
87 52
88 15
89 15
90 13
91 1
92 1
57 45
58 248
59 217
60 277
61 240
62 315
63 390
64 486
65 476
66 446
67 532
68 542
69 524
70 485
71 478
72 380
73 359
74 349
75 353
76 349
77 311
78 365
79 294
80 274
81 271
82 164
83 172
84 108
85 57
86 64
87 43
88 12
89 19
90 12
91 2
92 1
57 60
58 211
59 221
60 242
61 254
62 364
63 403
64 484
65 470
66 454
67 460
68 515
69 505
70 489
71 469
72 388
73 369
74 381
75 367
76 359
77 299
78 354
79 300
80 276
81 288
82 188
83 147
84 120
85 68
86 79
87 46
88 13
89 11
90 5
91 1
57 77
58 220
59 227
60 275
61 265
62 312
63 365
64 468
65 512
66 480
67 537
68 524
69 539
70 493
71 485
72 390
73 393
74 349
75 328
76 355
77 316
78 343
79 326
80 291
81 232
82 145
83 126
84 103
85 50
86 55
87 36
88 14
89 14
90 12
91 3
57 67
58 204
59 223
60 227
61 230
62 368
63 373

/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The d

57 76
58 229
59 227
60 240
61 267
62 353
63 414
64 479
65 515
66 467
67 483
68 473
69 505
70 538
71 487
72 402
73 368
74 377
75 326
76 348
77 346
78 321
79 303
80 266
81 243
82 141
83 133
84 126
85 61
86 66
87 50
88 11
89 8
90 11
57 74
58 249
59 181
60 250
61 237
62 348
63 378
64 437
65 493
66 477
67 517
68 507
69 505
70 548
71 481
72 413
73 423
74 327
75 367
76 347
77 312
78 342
79 303
80 275
81 231
82 178
83 154
84 88
85 69
86 79
87 48
88 8
89 12
90 2
57 69
58 213
59 213
60 232
61 236
62 337
63 385
64 504
65 474
66 459
67 521
68 520
69 517
70 517
71 461
72 432
73 388
74 349
75 325
76 336
77 333
78 349
79 304
80 269
81 254
82 172
83 157
84 121
85 75
86 64
87 40
88 13
89 15
90 6
57 79
58 209
59 217
60 252
61 239
62 305
63 397
64 452
65 530
66 493
67 476
68 529
69 538
70 510
71 447
72 390
73 398
74 372
75 369
76 339
77 301
78 324
79 292
80 270
81 248
82 170
83 138
84 152
85 76
86 72
87 44
88 13
89 8
90 8
91 2
92 1
57 61
58 245
59 217
60 218
61 278
62 322
63 441
64 451
65 466
66 466
67 4

/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The d

57 79
58 250
59 217
60 233
61 215
62 342
63 389
64 483
65 476
66 479
67 477
68 516
69 545
70 513
71 462
72 422
73 376
74 342
75 338
76 352
77 301
78 347
79 323
80 261
81 285
82 146
83 146
84 128
85 61
86 73
87 49
88 13
89 10
90 9
91 2
57 65
58 262
59 187
60 239
61 265
62 346
63 390
64 477
65 500
66 452
67 472
68 520
69 548
70 535
71 452
72 405
73 390
74 343
75 347
76 329
77 298
78 344
79 278
80 284
81 287
82 162
83 143
84 115
85 60
86 76
87 55
88 11
89 12
90 10
91 1
57 71
58 242
59 205
60 242
61 235
62 309
63 413
64 507
65 477
66 484
67 527
68 554
69 452
70 541
71 475
72 398
73 423
74 367
75 327
76 346
77 304
78 349
79 310
80 243
81 241
82 152
83 144
84 88
85 79
86 70
87 47
88 15
89 14
90 9
57 72
58 221
59 213
60 247
61 238
62 321
63 382
64 471
65 501
66 449
67 483
68 526
69 476
70 523
71 477
72 416
73 405
74 364
75 344
76 360
77 314
78 348
79 287
80 276
81 259
82 169
83 168
84 118
85 68
86 72
87 47
88 20
89 20
90 5
57 75
58 239
59 200
60 219
61 257
62 304
63 403
64 453
65 471
66 487
6

...

57 55
58 234
59 199
60 235
61 260
62 338
63 373
64 455
65 529
66 507
67 545
68 512
69 509
70 455
71 477
72 399
73 405
74 365
75 385
76 325
77 282
78 358
79 274
80 288
81 267
82 164
83 141
84 112
85 53
86 73
87 45
88 16
89 14
90 6
91 4
92 1
57 63
58 212
59 238
60 248
61 246
62 341
63 397
64 478
65 446
66 422
67 514
68 507
69 534
70 541
71 483
72 416
73 404
74 345
75 357
76 327
77 316
78 354
79 333
80 263
81 243
82 168
83 136
84 109
85 71
86 78
87 38
88 14
89 9
90 8
92 1
57 57
58 283
59 198
60 239
61 238
62 336
63 375
64 449
65 477
66 442
67 525
68 490
69 518
70 531
71 499
72 382
73 374
74 374
75 372
76 338
77 312
78 346
79 293
80 284
81 260
82 154
83 167
84 115
85 71
86 76
87 42
88 15
89 15
90 10
92 3
57 74
58 239
59 206
60 271
61 245
62 349
63 383
64 474
65 479
66 462
67 513
68 480
69 477
70 518
71 486
72 396
73 408
74 364
75 353
76 339
77 324
78 325
79 294
80 281
81 257
82 195
83 142
84 112
85 66
86 68
87 45
88 12
89 17
90 5
91 1
57 61
58 231
59 192
60 242
61 261
62 341
63 402
64 494


/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The d

57 67
58 242
59 222
60 239
61 248
62 346
63 338
64 470
65 485
66 472
67 489
68 559
69 541
70 495
71 460
72 431
73 357
74 350
75 375
76 335
77 312
78 356
79 330
80 259
81 255
82 167
83 128
84 114
85 65
86 77
87 37
88 19
89 11
90 8
91 1
57 73
58 232
59 204
60 227
61 237
62 334
63 383
64 496
65 483
66 488
67 518
68 489
69 481
70 464
71 464
72 417
73 439
74 395
75 351
76 369
77 305
78 327
79 300
80 275
81 267
82 165
83 135
84 119
85 59
86 77
87 53
88 17
89 11
90 5
91 1
57 58
58 234
59 204
60 265
61 238
62 318
63 433
64 464
65 477
66 465
67 519
68 491
69 498
70 490
71 474
72 410
73 385
74 364
75 327
76 337
77 314
78 365
79 302
80 300
81 247
82 186
83 171
84 117
85 55
86 72
87 43
88 14
89 14
90 8
92 1
57 67
58 235
59 200
60 240
61 232
62 321
63 409
64 514
65 498
66 445
67 474
68 559
69 491
70 501
71 468
72 426
73 408
74 340
75 331
76 335
77 302
78 337
79 336
80 273
81 245
82 200
83 146
84 114
85 65
86 73
87 45
88 10
89 9
90 9
92 2
57 56
58 224
59 196
60 266
61 228
62 349
63 383
64 476
65 465

/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/survival.py:300: FutureWarning: The d

<Figure size 640x480 with 0 Axes>